## Import Bibliotek

In [16]:
import numpy as np

## Wczytanie danych

In [17]:
nazwa_pliku = 'input.txt'

# Funkcja, która zastępuje przecinki jako separator dziesiętny na kropki - w celu poprawnego wczytania danych za
# pomocą np.loadtxt
def replace_commas_with_periods(filename):
    with open(filename, 'r') as file:
        contents = file.read()
        new_contents = contents.replace(',', '.')
    with open(filename, 'w') as file:
        file.write(new_contents)

replace_commas_with_periods(nazwa_pliku)
coords = np.loadtxt('./' + nazwa_pliku, delimiter=' ')
print("Dane wejściowe:")
print("   x,   f(x)")
print(coords)

Dane wejściowe:
   x,   f(x)
[[-2.5  16.5 ]
 [-2.   19.  ]
 [-1.5  17.65]
 [-1.   13.8 ]
 [-0.5   8.8 ]
 [ 0.    4.  ]
 [ 0.5   0.75]]


## Przypisanie danych do zmiennych

In [18]:
xi = coords[:, 0]
fx = coords[:, 1]
h = 0.5 # Odległośc między mierzonymi punktami

## Metoda Różnic zwykłych

### Funkcja która oblicza wartość następnej kolumny na podstawie danych z poprzedniej

W każdej kolejnej kolumnie oznaczonej jako delta f(x) obliczamy różnicę między kolejnymi wartościami z poprzedniej kolumny.
Funkcja poniżej zwraca numpy array z wartościami kolejnej kolumny.

![alt text](./assets/images/1.png "Title")

In [19]:
def get_next_column(base_column: np.ndarray):
   next_column = np.array([])
   # Nie chcemy obliczać różnicy między ostatnią wartością a wartością o jeden większą, ponieważ nie ma takiej wartości
   last_f = base_column[-1]
   for index, f in np.ndenumerate(base_column):
    if f != last_f:
        next_column = np.append(next_column, round((base_column[index[0] + 1] - f), 2))
   return next_column
print("Sprawdzenie poprawności działania funkcji:")
print(get_next_column(fx))

Sprawdzenie poprawności działania funkcji:
[ 2.5  -1.35 -3.85 -5.   -4.8  -3.25]


### Funkcja która oblicza wszystkie kolumny w tabeli

Na podstawie poprzedniej funkcji, możemy obliczyć kolejne kolumny w tabeli różnic zwykłych.
Funkcja będzie uruchamiać się rekurencyjnie, aż do momentu, kiedy wszystkie wartości w kolumnie będą takie same.
Następnie zwróci listę z wszystkimi kolumnami.

In [20]:
def calculate_cols(column: np.ndarray, columns: list = []) -> list:
    col = get_next_column(column)
    columns.append(col)
    if len(np.unique(col)) != 1:
        calculate_cols(col, columns)
    return columns

columns = calculate_cols(fx)
# Sprawdzenie poprawności działania funkcji
for i in range(len(columns)):
    print(f'Column {i+1}: {columns[i]}')

Column 1: [ 2.5  -1.35 -3.85 -5.   -4.8  -3.25]
Column 2: [-3.85 -2.5  -1.15  0.2   1.55]
Column 3: [1.35 1.35 1.35 1.35]


### Funkcja która oblicza pochodną w punkcie metodą różnicy zwykłej

In [21]:
def calculate_derivative_at_point(columns: list, x: int):
    # W funkcji podajemy wartość x dla jakiej będziemy obliczać pochodną, w funkcji skorzystać musimy z indeksu tejże wartości
    index_of_value_x = np.where(xi == x)[0]
    point_fs = []
    for column in columns:
        try:
            point_fs.append(column[index_of_value_x])
        except IndexError:
            pass
            # print("No element at index: ", x)

    for index, delta_f in enumerate(point_fs):
        skladnik = 1/(index+1) * delta_f
        skladnik = skladnik if index % 2 == 0 else -skladnik
        point_fs[index] = skladnik

    point_fs = np.array(point_fs)
    return (1/h * point_fs.sum())

for x in xi:
    print(f'Pochodna w punkcie {x}: ', calculate_derivative_at_point(columns, x))

Pochodna w punkcie -2.5:  9.75
Pochodna w punkcie -2.0:  0.6999999999999998
Pochodna w punkcie -1.5:  -5.65
Pochodna w punkcie -1.0:  -9.299999999999999
Pochodna w punkcie -0.5:  -11.15
Pochodna w punkcie 0.0:  -6.5
Pochodna w punkcie 0.5:  0.0


# Metoda różnicy wstecznej

W każdej kolejnej kolumnie oznaczonej jako delta f(x) obliczamy różnicę między kolejnymi wartościami z poprzedniej kolumny.
Funkcja poniżej zwraca numpy array z wartościami kolejnej kolumny.
![alt text](./assets/images/2.png "Title")

In [22]:
fx_reverse = np.flip(fx)

def get_next_column_reverse(base_column: np.ndarray):
   next_column = np.array([])
   # Nie chcemy obliczać różnicy między ostatnią wartością a wartością o jeden większą, ponieważ nie ma takiej wartości
   first_f = base_column[-1]
   for index, f in np.ndenumerate(base_column):
    if f != first_f:
        next_column = np.append(next_column, round((f - base_column[index[0] + 1]), 2))
   return next_column
print("Sprawdzenie poprawności działania funkcji:")
print(get_next_column_reverse(fx_reverse))

Sprawdzenie poprawności działania funkcji:
[-3.25 -4.8  -5.   -3.85 -1.35  2.5 ]


### Funkcja która oblicza wszystkie kolumny w tabeli

Na podstawie poprzedniej funkcji, możemy obliczyć kolejne kolumny w tabeli różnic zwykłych.
Funkcja będzie uruchamiać się rekurencyjnie, aż do momentu, kiedy wszystkie wartości w kolumnie będą takie same.
Następnie zwróci listę z wszystkimi kolumnami.

In [23]:
def calculate_cols_reverse(column: np.ndarray, columns: list = []) -> list:
    col = get_next_column_reverse(column)
    columns.append(col)
    if len(np.unique(col)) != 1:
        calculate_cols_reverse(col, columns)
    return columns

columns_reverse = calculate_cols_reverse(fx_reverse)
# Sprawdzenie poprawności działania funkcji
for i in range(len(columns_reverse)):
    print(f'Column {i+1}: {columns_reverse[i]}')

Column 1: [-3.25 -4.8  -5.   -3.85 -1.35  2.5 ]
Column 2: [ 1.55  0.2  -1.15 -2.5  -3.85]
Column 3: [1.35 1.35 1.35 1.35]


### Funkcja która oblicza pochodną w danym punkcie metodą różnicy wstecznej

In [24]:
def calculate_derivative_at_point_reverse(columns: list, x: int):
    # W funkcji podajemy wartość x dla jakiej będziemy obliczać pochodną, w funkcji skorzystać musimy z indeksu tejże wartości
    xi_reversed = np.flip(xi)
    index_of_value_x = np.where(xi_reversed == x)[0]
    point_fs = []
    for column in columns:
        try:
            point_fs.append(column[index_of_value_x])
        except IndexError:
            pass
            # print("No element at index: ", x)

    for index, delta_f in enumerate(point_fs):
        skladnik = 1/(index+1) * delta_f
        point_fs[index] = skladnik

    point_fs = np.array(point_fs)
    return (1/h * point_fs.sum())

for x in xi:
    print(f'Pochodna w punkcie {x}: ', calculate_derivative_at_point_reverse(columns_reverse, x))

Pochodna w punkcie -2.5:  0.0
Pochodna w punkcie -2.0:  5.0
Pochodna w punkcie -1.5:  -6.550000000000001
Pochodna w punkcie -1.0:  -9.299999999999999
Pochodna w punkcie -0.5:  -10.25
Pochodna w punkcie 0.0:  -8.5
Pochodna w punkcie 0.5:  -4.05


# Metoda różnicy centralnej

### Funkcja któa oblicza pośrednie wartości dla x
Pośrednie wartości to wartości, które znajdują się pomiędzy wartościami z tablicy xi.
Np. dla tablicy xi = [2.5, 2, 1.5] pośrednie wartości to [2.25, 1.75, 1.25]

In [25]:
def calculate_central_x(xi: np.ndarray):
    global h
    new_arr = []
    for x in xi:
        new_arr.append(x)
        new_arr.append(x + 1/2 * h)
    new_arr = np.delete(new_arr, -1)
    return new_arr

central_x = calculate_central_x(xi)
print(central_x)

[-2.5  -2.25 -2.   -1.75 -1.5  -1.25 -1.   -0.75 -0.5  -0.25  0.    0.25
  0.5 ]


### Funkcja która tworzy kolumnę z pustymi wartościami pomiędzy wartościami z tablicy xi

In [26]:
def create_column_with_empty_values_between(fx: np.ndarray):
    nan_array = np.full(len(fx),np.nan)
    calculated_array = np.array([])
    for element in zip(fx, nan_array):
        calculated_array = np.append(calculated_array, element)
    calculated_array = np.delete(calculated_array, -1)
    return calculated_array

print(create_column_with_empty_values_between(fx))

[16.5    nan 19.     nan 17.65   nan 13.8    nan  8.8    nan  4.     nan
  0.75]


### Funkcja która oblicza wszystkie kolumny w tabeli różnic centralnych

![alt text](./assets/images/3.png "Title")

In [27]:
def calculate_columns_central(columns: np.ndarray):
    columns_with_empty_values = []
    for index, col in enumerate(columns):
        c = create_column_with_empty_values_between(col)
        # Poprzednia funkcja oblicza i wstawia puste wartości tylko pomiędzy wartościami już istniejącymi
        # Tu dodajemy jeszcze po i*index pustych miejsc na początku i końcu tabeli, by zachować kształt tabeli
        for i in range(index+1):
            c = np.append(c, np.nan)
            c = np.insert(c, 0, np.nan)
        columns_with_empty_values.append(c)
    return columns_with_empty_values

# Tworzymy całość tablicy z puistymi wartościami zgodnie z wzorem
central_columns = np.vstack([create_column_with_empty_values_between(fx), calculate_columns_central(columns)])
print(central_columns)

[[16.5    nan 19.     nan 17.65   nan 13.8    nan  8.8    nan  4.     nan
   0.75]
 [  nan  2.5    nan -1.35   nan -3.85   nan -5.     nan -4.8    nan -3.25
    nan]
 [  nan   nan -3.85   nan -2.5    nan -1.15   nan  0.2    nan  1.55   nan
    nan]
 [  nan   nan   nan  1.35   nan  1.35   nan  1.35   nan  1.35   nan   nan
    nan]]


### Obliczanie zgodnie ze wzorem:

![alt text](./assets/images/4.png "Title")
![alt text](./assets/images/5.png "Title")

In [28]:
def calculate_derivative_at_point_central(columns: list, x: int):
    # W funkcji podajemy wartość x dla jakiej będziemy obliczać pochodną, w funkcji skorzystać musimy z indeksu tejże wartości
    index_of_value_x = np.where(central_x == x)[0]
    new_sum = []
    # Sztywne wstawki do poszczególnych elementów
    wstawki = [1/1, -1/6, 1/30]
    for index, col in enumerate(central_columns):
        if np.isnan(col[index_of_value_x]):
            try:
                df = (col[index_of_value_x-1] + col[index_of_value_x+1])/2
                new_sum.append(df)
            except IndexError:
                new_sum.append(np.nan)
    try:
        for index, wstawka in enumerate(wstawki):
            new_sum[index] = new_sum[index] * wstawka
    except IndexError:
        pass
    new_sum = (1/h) * np.array(new_sum).sum()
    return new_sum

for x in xi:
    d = calculate_derivative_at_point_central(columns, x)
    if not np.isnan(d):
        print(f'Pochodna w punkcie {x}: ', calculate_derivative_at_point_central(columns, x))

Pochodna w punkcie -1.5:  -5.65
Pochodna w punkcie -1.0:  -9.299999999999999
Pochodna w punkcie -0.5:  -10.25


## Porównanie wyników

In [32]:
import pandas as pd
from IPython.display import display

roznice_zwykle = np.array([0,0])
for x in xi:
    roznice_zwykle = np.vstack((roznice_zwykle, (x, calculate_derivative_at_point(columns, x))))
roznice_zwykle = np.delete(roznice_zwykle, 0, 0)

roznice_wsteczne =  np.array([0,0])
for x in xi:
    roznice_wsteczne = np.vstack((roznice_wsteczne, (x, calculate_derivative_at_point_reverse(columns_reverse, x))))
roznice_wsteczne = np.delete(roznice_wsteczne, 0, 0)

roznice_centralne = np.array([0,0])
for x in xi:
    roznice_centralne = np.vstack((roznice_centralne, (x, calculate_derivative_at_point_central(columns, x))))
roznice_centralne = np.delete(roznice_centralne, 0, 0)

pd_roznice_zwykle = pd.DataFrame(roznice_zwykle, columns=['x', 'Pochodna'])
pd_roznice_wsteczne = pd.DataFrame(roznice_wsteczne, columns=['x', 'Pochodna'])
pd_roznice_centralne = pd.DataFrame(roznice_centralne, columns=['x', 'Pochodna'])
pd_roznice_centralne = pd_roznice_centralne.dropna()

display(pd_roznice_zwykle)
display(pd_roznice_wsteczne)
display(pd_roznice_centralne)

,x,Pochodna
0,-2.5,9.75
1,-2.0,0.70
2,-1.5,-5.65
3,-1.0,-9.30
4,-0.5,-11.15
5,0.0,-6.50
6,0.5,0.00


,x,Pochodna
0,-2.5,0.00
1,-2.0,5.00
2,-1.5,-6.55
3,-1.0,-9.30
4,-0.5,-10.25
5,0.0,-8.50
6,0.5,-4.05


,x,Pochodna
2,-1.5,-5.65
3,-1.0,-9.30
4,-0.5,-10.25
